In [ ]:
import os

import cv2
import pandas as pd
import numpy as np

from skimage.feature import local_binary_pattern

Paths are initialized

In [ ]:
#wikiart (large dataset) will be added later

kaggle_train_path = "../data/train"
kaggle_test_path = "../data/test"

features_path = "../features"
painters_path = "../data/artists.csv"

kaggle_painter_names = os.listdir(kaggle_train_path)
painters = pd.read_csv(painters_path)
painters = painters["name"]

Feature 1: Average of HOG

In [ ]:
#Computing HOG over an image
def compute_hog(image):
    hog_desc = cv2.HOGDescriptor()
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hog_features = []

    for channel in range(hsv_image.shape[2]): #computing for HSV channels
        hog_features_channel = hog_desc.compute(hsv_image[:, :, channel])
        hog_features.append(hog_features_channel)

    return np.concatenate(hog_features)

#Computing average of HOG for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]

    hog_features_list = []
    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        hog_features = compute_hog(image)
        hog_features_list.append(hog_features)

    hog_features_array = np.array(hog_features_list)
    avg_hog_features = np.mean(hog_features_array, axis=0)

Feature 2: Average of LBP

In [ ]:
def compute_lbp(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    gray_image = hsv_image[:, :, 2] #to obtain texture information

    radius = 3
    n_points = 8 * radius
    features = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    lbp_hist, _ = np.histogram(features.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist /= np.sum(lbp_hist)
    return lbp_hist

average_lbp_features = {}

#Computing average of LBP for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]
    painter_lbp_features = []

    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        lbp_feature = compute_lbp(image)
        painter_lbp_features.append(lbp_feature)

    average_lbp_features[painter] = np.mean(painter_lbp_features, axis=0)

Feature 3: Standard Deviation of HOG

In [ ]:
#Computing standard deviation of HOG for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]

    hog_features_list = []
    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        hog_features = compute_hog(image)
        hog_features_list.append(hog_features)

    hog_features_array = np.array(hog_features_list)
    avg_hog_features = np.std(hog_features_array, axis=0)

Feature 4: Standard Deviation of LBP

In [ ]:
#Computing standard deviation of LBP for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]
    painter_lbp_features = []

    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        lbp_feature = compute_lbp(image)
        painter_lbp_features.append(lbp_feature)

    average_lbp_features[painter] = np.std(painter_lbp_features, axis=0)

Feature 5: Average of Gray Histogram

In [ ]:
def compute_gray_histogram(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, _ = np.histogram(gray_image.ravel(), bins=256, range=[0,256])
    return hist

avg_gray_hist = {}

for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    painter_gray_hist = []

    for image_name in os.listdir(painter):
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        gray_hist = compute_gray_histogram(image)
        painter_gray_hist.append(gray_hist)

    avg_gray_hist[painter] = np.mean(painter_gray_hist, axis=0)

Feature 6: Average of Image Edges Histogram

In [ ]:
def compute_edge_histogram(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150)
    hist, _ = np.histogram(edges.ravel(), bins=256, range=[0,256])
    return hist

avg_edge_hist = {}

for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    painter_edge_hist = []

    for image_name in os.listdir(paintings_path):
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        edge_hist = compute_edge_histogram(image)
        painter_edge_hist.append(edge_hist)

    avg_edge_hist[painter] = np.mean(painter_edge_hist, axis=0)